In [ ]:
#https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b
# Running the version as 1.x is optional, without that first line it will run the last version of tensorflow for Colab.

#import keras
import tensorflow as tf 
from tensorflow import keras
import tensorflow.keras as K

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D

import pandas as pd
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
#from keras.utils import np_utils
from keras import utils as np_utils
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window




In [ ]:
# Load data
# CIFAR-10 is a dataset with 60000 32x32 colour images grouped in 10 classes, that means 6000 images per class. 
# This is a dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 10 categories.
# The categories are airplane, automobile, beer, cat, deer, dog, frog, horse, ship, truck. 
# We can take advantage of the fact that these categories and a lot more are into the Imagenet collection.
# Loading the CIFAR-10 datasets
from keras.datasets import cifar10


In [ ]:
# Preprocess data function
# Now that the data is loaded, we are going to build a preprocess function for the data. 
# We have X as a numpy array of shape (m, 32, 32, 3) where m is the number of images, 
# 32 and 32 the dimensions, and 3 is because we use colored images. 
# We have a set of X for training and a set of X for validation. 
# Y is a numpy array of shape (m, ) that we want to be our labels. 
# Since we work with 10 different categories, we make use of one-hot encoding with a 
# function of Keras that makes our Y into a shape of (m, 10). That also applies for the validation.

def preprocess_data(X,Y):
  X_p = K.applications.xception.preprocess_input(X)
  Y_p = K.utils.to_categorical(Y,10)
  return X_p, Y_p
  

In [ ]:
# load and split data
# The data, split between train and test sets:
# (x_train, y_train), (x_test, y_test) = K.datasets.cifar100.load_data()
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_rows, img_cols = 32, 32
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)


x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [ ]:
# Preprocess data
# Next, we are going to call our function with the parameters loaded from the Cifar 10 database.

x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 10)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 10)


In [ ]:
# Using weights of a trained neural network
# A pretrained model from the Keras Applications has the advantage of allowing you to use weights that
# are already calibrated to make predictions. In this case, we use the weights from Imagenet 
# and the network is a Xception. The option include_top=False allows feature extraction by removing 
# the last dense layers. This let us control the output and input of the model.


input_t = K.Input(shape=(128,128,3))
# input_t = K.Input(shape=(224,224,3))
# input_t = K.Input(shape=(32,32,3))
xception_model = K.applications.Xception(include_top=False,
                                    weights="imagenet",
                                    input_tensor=input_t)


In [ ]:
# In this case, we ‘freeze’ all layers except for the last few block of the Exception.

for layer in xception_model.layers[:95]:
  layer.trainable=False
  

In [ ]:
# We can check that we did it correctly with:
# False means that the layer is ‘freezed’ or is not trainable and 
# True that when we run our model, the weights are going to be adjusted.

for i, layer in enumerate(xception_model.layers):
  print(i,layer.name,"-",layer.trainable)
  

0 input_4 - False
1 block1_conv1 - False
2 block1_conv1_bn - False
3 block1_conv1_act - False
4 block1_conv2 - False
5 block1_conv2_bn - False
6 block1_conv2_act - False
7 block2_sepconv1 - False
8 block2_sepconv1_bn - False
9 block2_sepconv2_act - False
10 block2_sepconv2 - False
11 block2_sepconv2_bn - False
12 conv2d_12 - False
13 block2_pool - False
14 batch_normalization_21 - False
15 add_36 - False
16 block3_sepconv1_act - False
17 block3_sepconv1 - False
18 block3_sepconv1_bn - False
19 block3_sepconv2_act - False
20 block3_sepconv2 - False
21 block3_sepconv2_bn - False
22 conv2d_13 - False
23 block3_pool - False
24 batch_normalization_22 - False
25 add_37 - False
26 block4_sepconv1_act - False
27 block4_sepconv1 - False
28 block4_sepconv1_bn - False
29 block4_sepconv2_act - False
30 block4_sepconv2 - False
31 block4_sepconv2_bn - False
32 conv2d_14 - False
33 block4_pool - False
34 batch_normalization_23 - False
35 add_38 - False
36 block5_sepconv1_act - False
37 block5_sepconv

In [ ]:
    # Add Flatten and Dense layers on top of Xception
    # Now, we need to connect our pretrained model with the new layers 
    # of our model. We can use global pooling or a flatten layer to connect 
    # the dimensions of the previous layers with the new layers. 
    
    to_res = (128, 128)
    # to_res = (224, 224)
    # to_res = (32, 32)
    model = K.models.Sequential()
    model.add(K.layers.Lambda(lambda image: tf.image.resize(image, to_res))) 
    model.add(xception_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(256, activation='relu'))
    model.add(K.layers.Dropout(0.1))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(128, activation='relu'))
    model.add(K.layers.Dropout(0.1))
    model.add(K.layers.BatchNormalization())
   # model.add(K.layers.Dense(64, activation='relu'))
   # model.add(K.layers.Dropout(0.1))
   # model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(10, activation='softmax'))

In [ ]:
# Compile model and train
# Results
# In the 8th epoch, the values are very similar and it is interesting to note that 
# in the first validation accuracy is higher than training. 
# This is because of dropout use, which in Keras, it has a different behavior 
# for training and testing. In testing time, all the features are ready and 
# the dropout is turned off, resulting in a better accuracy. 
# This readjust on the last epochs since the model continues changing on the training.

model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=1,
                        validation_data=(x_test, y_test)
                       )
model.summary()


Epoch 1/5
1563/1563 [==============================] - 136s 84ms/step - loss: 0.5330 - accuracy: 0.8301 - val_loss: 0.3460 - val_accuracy: 0.8890
Epoch 2/5
1563/1563 [==============================] - 129s 83ms/step - loss: 0.2874 - accuracy: 0.9086 - val_loss: 0.3376 - val_accuracy: 0.8945
Epoch 3/5
1563/1563 [==============================] - 129s 83ms/step - loss: 0.1964 - accuracy: 0.9365 - val_loss: 0.2914 - val_accuracy: 0.9087
Epoch 4/5
1563/1563 [==============================] - 136s 87ms/step - loss: 0.1415 - accuracy: 0.9541 - val_loss: 0.3027 - val_accuracy: 0.9141
Epoch 5/5
1563/1563 [==============================] - 130s 83ms/step - loss: 0.1063 - accuracy: 0.9661 - val_loss: 0.3070 - val_accuracy: 0.9129
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_3 (Lambda)           (None, 128, 128, 3)       0         
                                                  

## Changes in Final Model:
- Dropout reduced to **0.1** <br>
- Img Size increased to **224 x 224 x 3** <br>
- Optimizer changed to **"adam"** <br>
- No. of layers frozen increased from 75 to **95** <br>